# Entities as Experts

This notebook is a code implementation of the paper "Entities as Experts: Sparse Memory Access with Entity Supervision" by Févry, Baldini Soares, FitzGerald, Choi, Kwiatowski.

## Problem definition and high-level model description

We want to perform question answering on typical one-shot questions that require external knowledge or context. For example, in order to answer the question "Which country was Charles Darwin born in?" one needs some text providing answers on typical structured scenarios.

In this case, however, we want to rely on knowledge-graph extracted information. For example, in the question given here, we can prune out unrelated to the antropologist and evolution theorist Charles Darwins, e.g. Charles River, Darwin City etc. 

In the paper, the authors propose to augment BERT in the task of cloze-type question answering by leveraging an Entity Memory extracted from e.g. a Knoweldge Graph.

![Entity as Experts description](images/eae_highlevel.png)

The Entity Memory is a simple bunch of embeddings of entities extracted from a Knowledge Graph. Relationships are ignored (see the Facts as Experts paper and notebook to see how they could be used).

## Datasets

> We assume access to a corpus $D={(xi,mi)}$,where all entity mentions are detected but not necessarily  all  linked  to  entities.   We  use  English Wikipedia as our corpus, with a vocabulary of 1m entities. Entity links come from hyperlinks, leading to 32m 128 byte contexts containing 17m entity links.

In the appendix B, it is explained that:

> We build our training corpus of contexts paired with entity mention labels from the 2019-04-14 dump of English Wikipedia. We first divide each article into chunks of 500 bytes,resulting in a corpus of 32 million contexts withover 17 million entity mentions. We restrict our-selves  to  the  one  million  most  frequent  entities
(86% of the linked mentions).

Given that the dump 2019-04-14 is not available at the time of writing, we will adopt the revision 2020-11-01.

Entities are thus partially extracted by link annotations (e.g. they associate with each token a mention if that token belongs to a wikipedia url).

## Mention Detection

> In addition to the Wikipedia links, we annotaten each sentence with unlinked mention spans using the mention detector from Section 2.2

The mention detection head discussed in Section 2.2 is a simple BIO sequence: each token is annotated with a B (beginning), I (inside) or O (outside) if they are respectivelly beginning, inside or outside of a mention. The reason why we use both BIO and EL is to avoid inconsistencies.

There is a catch. In the paper, they explain they used Google NLP APIs to perform entity detection and linking on large-scale Wikipedia entries, that is, to have a properly annotated Wikipedia dataset.

Since we cannot technically afford this, we will use spacy's entity detection and linking capabilities as a baseline. Data quality 

## Chunking

- Split articles by chunks of 500 bytes (assuming unicode encoding).
- We will elide sentences till the last period to make sure they reach such limit without giving weird effects.

## Tokenization:

- BERT Tokenizer (e.g. Wordpiece) using lowercase vocabulary, limited to 128 distinct word-piece tokens.

In [125]:
from tools.providers import WikipediaProvider

#WikipediaProvider.dump_full_dataset(revision="20201101")
from trec_car import read_data
from tools.dumps import wrap_open
from collections import defaultdict
import spacy

from trec_car.read_data import Page, Section, List, Para, ParaLink, ParaText, ParaBody

nlp = spacy.load("en_core_web_md", pipeline=["tokenizer"])

In [126]:
def handle_section(skel, toks, links):
    for subskel in skel.children:
        visit_section(subskel, toks, links)

def handle_list(skel, toks, links):
    visit_section(skel.body, toks, links)

def handle_para(skel: Para, toks, links):
    paragraph = skel.paragraph
    bodies = paragraph.bodies

    for body in bodies:
        visit_section(body, toks, links)

def handle_paratext(body: ParaBody, toks, links):
    lemmas = [x.text for x in nlp(body.get_text())]
    toks.extend(lemmas)
    links.extend(["PAD"] * len(lemmas))
    pass

def handle_paralink(body: ParaLink, toks, links):
    lemmas = [x.text for x in nlp(body.anchor_text)]
    toks.extend(lemmas)
    links.extend([body.page] + ["PAD"] * (len(lemmas) - 1))
    pass

def nothing():
    return lambda body, toks, links: None

handler = defaultdict(nothing, {Section: handle_section,
                     Para: handle_para,
                     List: handle_list,
                     ParaLink: handle_paralink,
                     ParaText: handle_paratext})


def visit_section(skel, toks, links):
    # Recur on the sections
    handler[type(skel)](skel, toks, links)

In [128]:
with wrap_open("wikipedia/car-wiki2020-01-01/enwiki2020.cbor", "rb") as toc:
    for idx, page in enumerate(read_data.iter_annotations(toc)):
        links = []
        toks = []
        for skel in page.skeleton:
            visit_section(skel, toks, links)
        #print(links)
        break

In [ ]:
import torch
from torch.utils.data import Dataset, 
import tqdm
from transformers import BertTokenizer, BertConfig
from tools.dumps import get_filename_path

from keras.preprocessing.sequence import pad_sequences

import random

from trec_car.read_data import AnnotationsFile, ParagraphsFile

class WikipediaCBOR(Dataset):
    """
    This is a simple CBOR loader.
    """
    def __init__(self, cbor_path):
        """
        :param cbor_path the path of the wikipedia cbor export
        """
        # Let trec deal with that in my place
        
        self.cbor_path = get_filename_path(cbor_path)
        self.cbor_toc_annotations = AnnotationsFile(self.cbor_path)
        self.cbor_toc_paragraphs = ParagraphsFile(self.cbor_path)
        
        self.keys = selfself.cbor_toc_paragraphs.keys()
        
        self.tokenizer = BertTokenizer.from_pretrained('bert-base-cased')
        
        # preprocess and find the top k unique wikipedia links
        
    
        
    def __len__(self):
        return len(self.keys)
    
    def __get_whole_text(paragraph):
        # Tokenize by word. When a link is detected, 
    
    def __getitem__(self, idx):
        if torch.is_tensor(idx)
            idx = idx.tolist()
    
        paragraphs = [self.cbor_toc_paragraphs.get(self.keys[i]) for i in idx]
                
        # TODO: can we parallelize this?
        
        
    
    return torch.tensor(paragraphs)
    
        

In [93]:
with wrap_open("wikipedia/car-wiki2020-01-01/enwiki2020.cbor", "rb") as toc:
    #toc.seek(46102636493)
    
    for idx, page in enumerate(read_data.iter_pages(toc)):
        if idx >= 10:
            break
        print(page)
        page.

Page(Anarchism)
Page(Albedo)
Page(Autism)
Page(A)
Page(Achilles)
Page(Alabama)
Page(Abraham Lincoln)
Page(An American in Paris)
Page(Aristotle)
Page(Academy Award for Best Production Design)


## Model

In the paper, the authors explain they used a modified BERT.

In [2]:
from zipfile import ZipFile
from tools.dumps import wrap_open
import pandas as pd
from tqdm import tqdm, trange

!cd data && unzip ner.csv.zip

Archive:  ner.csv.zip
replace ner.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: ^C


In [3]:
# The columns are a bit irregular.
names = []
with wrap_open("ner.csv", "r", encoding="latin1") as f:
    names = f.readline().strip().split(",")[1:]
    names = names + list(range(34 - len(names)))

print(names)

with wrap_open("ner.csv", "rb") as f:
    data = pd.read_csv(f, encoding="latin1", names=names).fillna(method="ffill")
data.tail(10)

['lemma', 'next-lemma', 'next-next-lemma', 'next-next-pos', 'next-next-shape', 'next-next-word', 'next-pos', 'next-shape', 'next-word', 'pos', 'prev-iob', 'prev-lemma', 'prev-pos', 'prev-prev-iob', 'prev-prev-lemma', 'prev-prev-pos', 'prev-prev-shape', 'prev-prev-word', 'prev-shape', 'prev-word', 'sentence_idx', 'shape', 'word', 'tag', 0, 1, 2, 3, 4, 5, 6, 7, 8, 9]


/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:3072: DtypeWarning: Columns (21,25,26,27,28,29,30,31,32,33) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,lemma,next-lemma,next-next-lemma,next-next-pos,next-next-shape,next-next-word,next-pos,next-shape,next-word,pos,...,0,1,2,3,4,5,6,7,8,9
1050787,1048565.0,impact,.,__end1__,__END1__,wildcard,__END1__,.,punct,.,...,O,prev-prev-pos,prev-prev-shape,prev-prev-word,prev-shape,prev-word,sentence_idx,shape,word,tag
1050788,1048566.0,.,__end1__,__end2__,__END2__,wildcard,__END2__,__END1__,wildcard,__END1__,...,O,prev-prev-pos,prev-prev-shape,prev-prev-word,prev-shape,prev-word,sentence_idx,shape,word,tag
1050789,1048567.0,indian,forc,said,VBD,lowercase,said,NNS,lowercase,forces,...,B-gpe,prev-prev-pos,prev-prev-shape,prev-prev-word,prev-shape,prev-word,sentence_idx,shape,word,tag
1050790,1048568.0,forc,said,they,PRP,lowercase,they,VBD,lowercase,said,...,O,prev-prev-pos,prev-prev-shape,prev-prev-word,prev-shape,prev-word,sentence_idx,shape,word,tag
1050791,1048569.0,said,they,respond,VBD,lowercase,responded,PRP,lowercase,they,...,O,prev-prev-pos,prev-prev-shape,prev-prev-word,prev-shape,prev-word,sentence_idx,shape,word,tag
1050792,1048570.0,they,respond,to,TO,lowercase,to,VBD,lowercase,responded,...,O,prev-prev-pos,prev-prev-shape,prev-prev-word,prev-shape,prev-word,sentence_idx,shape,word,tag
1050793,1048571.0,respond,to,the,DT,lowercase,the,TO,lowercase,to,...,O,prev-prev-pos,prev-prev-shape,prev-prev-word,prev-shape,prev-word,sentence_idx,shape,word,tag
1050794,1048572.0,to,the,attack,NN,lowercase,attack,DT,lowercase,the,...,O,prev-prev-pos,prev-prev-shape,prev-prev-word,prev-shape,prev-word,sentence_idx,shape,word,tag
1050795,1048573.0,the,attack,with,IN,lowercase,with,NN,lowercase,attack,...,O,prev-prev-pos,prev-prev-shape,prev-prev-word,prev-shape,prev-word,sentence_idx,shape,word,tag
1050796,1048574.0,attack,with,machine-gun,JJ,contains-hyphen,machine-gun,IN,lowercase,with,...,O,prev-prev-pos,prev-prev-shape,prev-prev-word,prev-shape,prev-word,sentence_idx,shape,word,tag


In [30]:
data.columns

Index([          'lemma',      'next-lemma', 'next-next-lemma',
         'next-next-pos', 'next-next-shape',  'next-next-word',
              'next-pos',      'next-shape',       'next-word',
                   'pos',        'prev-iob',      'prev-lemma',
              'prev-pos',   'prev-prev-iob', 'prev-prev-lemma',
         'prev-prev-pos', 'prev-prev-shape',  'prev-prev-word',
            'prev-shape',       'prev-word',    'sentence_idx',
                 'shape',            'word',             'tag',
                       0,                 1,                 2,
                       3,                 4,                 5,
                       6,                 7,                 8,
                       9],
      dtype='object')

In [4]:
def aggregate(s):
    return [(w, p, t) for w, p, t in zip(s["word"].values.tolist(),
                                               s["pos"].values.tolist(),
                                               s["tag"].values.tolist())]

sentences = [s for s in data.groupby("sentence_idx").apply(aggregate)]

In [5]:
sentences[2]

[('number', 'million', '89.7'),
 ('number', 'million', '94.8'),
 ('number', 'million', '118'),
 ('number', 'million', '130.6'),
 ('number', 'million', '18'),
 ('number', 'million', '34'),
 ('number', 'million', '128'),
 ('number', 'million', '133'),
 ('number', 'million', '130'),
 ('number', 'million', '135'),
 ('number', 'million', '722')]

In [6]:
utterances = [[w[0] for w in s] for s in sentences]
tags = [[w[1] for w in s] for s in sentences]
labels = [[w[2] for w in s] for s in sentences]

tag_values = list(set(data["tag"].values))
tag_values.append("PAD")
tag2idx = {t: i for i, t in enumerate(tag_values)}

In [7]:
import torch
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from transformers import BertTokenizer, BertConfig

from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split

In [8]:
MAX_LEN = 75 ## can replace with 512 as per the original paper
bs = 32

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
n_gpu = torch.cuda.device_count()
torch.cuda.get_device_name(0)

'GeForce RTX 2080 Ti'

In [9]:
tokenizer = BertTokenizer.from_pretrained('bert-base-cased', do_lower_case=False)

def tokenize_preserve_labels(sentence, text_labels):
    """
    Tokenize the given sentence. Extend the corresponding label
    for all the tokens the word is made of.
    
    Assumption: len(sentence) == len(text_labels)
    """
    
    tokenized_sentence = []
    labels = []
    
    for word, label in zip(sentence, text_labels):
        tokenized_word = tokenizer.tokenize(word)
        n_subwords = len(tokenized_word)
        
        tokenized_sentence.extend(tokenized_word)
        labels.extend([label] * n_subwords)
    
    return tokenized_sentence, labels

In [10]:
tokenized_texts_labels = [
    tokenize_preserve_labels(sent, labs) for sent, labs in zip(utterances, labels)
]

In [11]:
tokenized_texts = [token_label_pair[0] for token_label_pair in tokenized_texts_labels]
tokenized_labels = [token_label_pair[1] for token_label_pair in tokenized_texts_labels]

In [12]:
input_ids = pad_sequences([tokenizer.convert_tokens_to_ids(txt) for txt in tokenized_texts],
                                maxlen=MAX_LEN, dtype="long", value=0.0, truncating="post", padding="post")

In [13]:
tags = pad_sequences([[tag2idx.get(l) for l in lab] for lab in labels],
                         maxlen=MAX_LEN, value=tag2idx["PAD"], padding="post",
                         dtype="long", truncating="post")

In [14]:
# apply a mask to the padding only.
# Normally, BERT masks are used for cloze-style questions, but this is a translation problem
attention_masks = [[float(i != 0.0) for i in ii] for ii in input_ids]

## Setting up a dataloader

In [15]:
# Prepare train and validation sets

train_inputs, validation_inputs, train_tags, validation_tags = train_test_split(input_ids, tags, random_state=42, test_size=0.1)
train_masks, validation_masks, _, _ = train_test_split(attention_masks, input_ids, random_state=42, test_size=0.1)

train_inputs = torch.tensor(train_inputs)
validation_inputs = torch.tensor(validation_inputs)
train_tags = torch.tensor(train_tags)
validation_tags = torch.tensor(validation_tags)
train_masks = torch.tensor(train_masks)
validation_masks = torch.tensor(validation_masks)

In [16]:
train_data = TensorDataset(train_inputs, train_masks, train_tags)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=bs)

validation_data = TensorDataset(validation_inputs, validation_masks, validation_tags)
validation_sampler = RandomSampler(validation_data)
validation_dataloader = DataLoader(validation_data, sampler=validation_sampler, batch_size=bs)

## Load and finetune the model

In [17]:
import transformers
from transformers import BertForTokenClassification, AdamW

transformers.__version__

'2.11.0'

In [18]:
model = BertForTokenClassification.from_pretrained(
    "bert-base-cased",
    num_labels=len(tag2idx),
    output_attentions = False,
    output_hidden_states = False
)                                                  

In [29]:
tokenized = tokenizer.encode("This is madness. Madness? THIS IS SPARTA!")
#output = model.forward(torch.tensor(tokenized).unsqueeze(0), labels=torch.tensor([[0] * 16]))


(tensor(10.3149, grad_fn=<NllLossBackward>), tensor([[[-1.8813e-01, -1.4686e-01,  1.0079e-01,  ...,  1.3702e-01,
           6.3725e-01, -1.9620e-01],
         [ 9.6525e-02,  6.8279e-04, -1.5830e-02,  ...,  2.0342e-01,
           2.9020e-01, -7.9644e-03],
         [ 3.0085e-02,  1.1656e-02,  4.1793e-02,  ...,  5.8692e-02,
          -1.8981e-02,  4.2294e-01],
         ...,
         [ 1.1023e-01, -1.2311e-01,  7.3450e-02,  ...,  1.6149e-01,
           1.2158e-01,  3.0233e-01],
         [ 1.3686e-01,  1.6160e-01,  3.9345e-02,  ..., -8.0749e-02,
          -9.8029e-02,  3.2206e-01],
         [ 9.9725e-01,  6.3929e-02,  6.3957e-01,  ..., -5.8454e-01,
          -2.2921e-02,  4.7836e-01]]], grad_fn=<AddBackward0>))


In [140]:
model.cuda()

BertForTokenClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(28996, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwis

ModuleAttributeError: 'BertForTokenClassification' object has no attribute 'loss'

In [141]:
FULL_FINETUNING = True

if FULL_FINETUNING:
    param_optimizer = list(model.named_parameters())
    no_decay = ['bias', 'gamma', 'beta']
    optimizer_grouped_parameters = [
        {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)],
         'weight_decay_rate': 0.01},
        {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)],
         'weight_decay_rate': 0.0}
    ]
else:
    param_optimizer = list(model.classifier.named_parameters())
    optimizer_grouped_parameters = [{"params": [p for n, p in param_optimizer]}]
    
    
optimizer = AdamW(
    optimizer_grouped_parameters,
    lr=3e-5,
    eps=1e-8
)

In [142]:
from transformers import get_linear_schedule_with_warmup

epochs = 3
max_grad_norm = 1.0

total_steps = len(train_dataloader) * epochs

scheduler = get_linear_schedule_with_warmup(
    optimizer,
    num_warmup_steps=0,
    num_training_steps=total_steps
)

In [143]:
from seqeval.metrics import f1_score, accuracy_score

In [164]:
import numpy as np

loss_values, validation_loss_values = [], []

for _ in trange(epochs, desc="Epoch"):
    model.train()
    total_loss = 0
    
    for step, batch in enumerate(train_dataloader):
        batch = tuple(t.to(device) for t in batch)
        b_input_ids, b_input_mask, b_labels = batch
        
        model.zero_grad()
        
        outputs = model(b_input_ids, token_type_ids=None,
                           attention_mask=b_input_mask, labels=b_labels)
    
        loss = outputs[0]

        loss.backward()
        total_loss += loss.item()
        torch.nn.utils.clip_grad_norm_(parameters=model.parameters(),
                                              max_norm=max_grad_norm)
        
        # update parameters, and choose next learning rate
        optimizer.step()
        scheduler.step()
    
    avg_train_loss = total_loss / len(train_dataloader)
    print("Average train loss: {}".format(avg_train_loss))
    
    loss_values.append(avg_train_loss)
    
    # Now evaluate on the validation set
    
    model.eval()
    eval_loss, eval_accuracy = 0.0, 0.0
    number_eval_steps, number_eval_examples = 0, 0
    predictions, true_labels = [], []
    
    for batch in validation_dataloader:
        batch = tuple(t.to(device) for t in batch)
        b_input_ids, b_input_mask, b_labels = batch
        
        
        with torch.no_grad():
            outputs = model(b_input_ids, token_type_ids=None,
                               attention_mask=b_input_mask, labels=b_labels)
        
        # Move logits and labels to cpu
        logits = outputs[1].detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()
        
        # Calculate the accuracy for this batch
        eval_loss += outputs[0].mean().item()
        
        predictions.extend([list(p) for p in np.argmax(logits, axis=2)])
        true_labels.extend(label_ids)
    
    
    eval_loss = eval_loss / len(validation_dataloader)
    validation_loss_values.append(eval_loss)
    pred_tags = [tag_values[p_i] for p, l in zip(predictions, true_labels)
                                          for p_i, l_i in zip(p, l)]
    true_tags = [tag_values[l_i] for l in true_labels
                                      for l_i in l]
    
    print("Validation Accuracy: {}".format(accuracy_score(pred_tags, true_tags)))
    print("Validation F1-Score: {}".format(f1_score(pred_tags, true_tags)))
    print()

Epoch:   0%|          | 0/3 [00:04<?, ?it/s]


KeyboardInterrupt: 

In [167]:
pred_tags = [[tag_values[p_i] for p_i, l_i in zip(p, l)]
                 for p, l in zip(predictions, true_labels)]
    
true_tags = [[tag_values[l_i] for l_i in l]
                 for l in true_labels]

print(pred_tags[0])
print(true_tags[0])

['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'PAD', 'PAD', 'PAD', 'PAD', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'PAD', 'PAD', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'PAD', 'PAD', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']
['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD']


In [12]:
from torch.nn import Module, Embedding, Dropout, ModuleList, Linear
import torch.nn as nn
import torch
import math

GELU = torch.nn.GELU
LayerNorm = torch.nn.LayerNorm

l0 = 4
l1 = 8

    
class EntityMemory(Module):
    """
    Entity Memory, as described in the paper
    """
    def __init__(self, embedding_size: int, entity_size: int,
                   entity_embedding_size: int):
        """
        :param embedding_size the size of an embedding. In the EaE paper it is called d_emb, previously as d_k
            (attention_heads * embedding_per_head)
        :param entity_size also known as N in the EaE paper, the maximum number of entities we store
        :param entity_embedding_size also known as d_ent in the EaE paper, the embedding of each entity
        
        """
        self.N = entity_size
        self.d_ent = entity_embedding_size
        self.w_f = Linear(d_ent, 2*embedding_size)
        
    def forward(self, x, entity_spans, num_entities, k=None):
        """
        :param x the (raw) output of the first transformer block. It has a shape:
                B x N x (embed_size)
        :param entity_spans entities and spans of such entities.
                Shape: B x C x 3. Each "row" contains a triple (e_k, s_mi, t_mi)
                where e_k is an (encoded) entity id, s_mi and t_mi are indices.
        :param num_entities the number of found entities for each batch.
        :param k the number of nearest entities to consider when softmax-ing.
                if k = None, all the entities are used.
                In the paper, one should set k for when running the inference
        """
        
        mentions = [entity_spans[:, :mentions_per_batch] for mentions_per_batch in num_entities]
        pass
        